In [ ]:
base_dir = '/data/coding/patent_qa/test/'
with open(base_dir+'test_finished_list.txt', 'r') as f:
    finished_text = f.read()
finished_files = finished_text.split('\n')

# 1. 存储为图片

In [ ]:
import fitz  # PyMuPDF
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
base_dir = '/data/coding/patent_qa/test/'
pdf_file_list  = [x for x in os.listdir(base_dir+'/documents/') if 'pdf' in x]

finished_list = []
for file_name in tqdm(pdf_file_list):
    if file_name in finished_files:
        continue
    pdf_document = fitz.open(base_dir+'/documents/'+file_name)
    os.makedirs(base_dir+'/pdf_img/'+file_name.split('.')[0],exist_ok=True)
    # 获取第一页
    for i in range(pdf_document.page_count):
        page = pdf_document.load_page(i)  # 注意：页码从0开始
        # 将页面转换为图像
        pix = page.get_pixmap(dpi=600) # 这些文档600的dpi够了
        pix.save(base_dir+'/pdf_img/'+file_name.split('.')[0]+'/'+str(i+1)+'.jpg')
    finished_list.append(file_name)
    # 完成的写入txt防止之后重复
    with open(base_dir+'test_finished_list_1.txt', 'a') as f:
        f.write(file_name + '\n')

In [ ]:
finished_files

# 2. OCR

In [ ]:
from swift.llm import InferEngine, InferRequest, PtEngine, RequestConfig, load_dataset
from swift.plugin import InferStats
from swift.llm import VllmEngine
import os
os.environ['VIDEO_MAX_PIXELS'] = '50176'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["MAX_PIXELS"] = "1229312"
os.environ['FPS_MAX_FRAMES'] = "2"
model_path = "/data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct/"
# 多卡设置tensor_parallel_size为卡数
engine = VllmEngine(model_path,model_type='qwen2_5_vl',gpu_memory_utilization=0.9,limit_mm_per_prompt={"image": 1},tensor_parallel_size=2)
def infer_batch(engine, infer_requests):
    request_config = RequestConfig(max_tokens=10240, temperature=0)
    metric = InferStats()
    resp_list = engine.infer(infer_requests, request_config)
    response = resp_list[0].choices[0].message.content
    return response

In [2]:
def qwen_ocr_cn(img_path):
    prompt="你是一个OCR专家，请提取以下图片中的所有文字内容，并原样返回。"
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": img_path,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]

    data = dict()
    data['messages'] = messages
    infer_requests = [InferRequest(**data)]
    response = infer_batch(engine, infer_requests)
    return response

def qwen_ocr_en(img_path):
    prompt="You are an OCR expert. Please extract all text from the following images, which have been converted from PDFs, and return the exact text content found in the attached image."
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": img_path,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]

    data = dict()
    data['messages'] = messages
    infer_requests = [InferRequest(**data)]
    response = infer_batch(engine, infer_requests)
    return response

In [ ]:
base_dir = '/data/coding/patent_qa/test/'
with open(base_dir+'test_ocr_finished_list_1.txt', 'r') as f:
    finished_text = f.read().split('\n')

In [ ]:
# 进行ocr
import fitz  # PyMuPDF
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
base_dir = '/data/coding/patent_qa/test/'
pdf_file_list = [x for x in os.listdir(base_dir+'/pdf_img/')]
# 前400个，排序
pdf_file_list = sorted(pdf_file_list)[:400]

for pdf_file in pdf_file_list:
    if pdf_file in finished_text:
        continue
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_img/'+file_name) if 'jpg' in x] # pdf文件存储的jpg
    os.makedirs(base_dir+'/pdf_ocr/'+file_name,exist_ok=True)
    for k in tqdm(range(len(file_list))):
        img_path = base_dir+'/pdf_img/'+file_name + '/' + file_list[k]
        page_num = int(file_list[k].split('.')[0])
        # ocr
        response = qwen_ocr_cn(img_path)
        # 保存ocr结果为txt
        with open(base_dir+f'/pdf_ocr/{file_name}/{page_num}.txt', 'w', encoding='utf-8') as text_file:
            text_file.write(response)
    print(file_name,'  ok')
    with open(base_dir+'test_ocr_finished_list_1.txt', 'a') as f:
        f.write(file_name + '\n')

In [ ]:
from transformers import AutoProcessor, AutoTokenizer

from vllm import LLM, SamplingParams
from vllm.multimodal.utils import fetch_image
from vllm.utils import FlexibleArgumentParser
import os
os.environ['VIDEO_MAX_PIXELS'] = '50176'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["MAX_PIXELS"] = "1229312"
os.environ['FPS_MAX_FRAMES'] = "2"
model_path = "/data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct/"
# 多卡设置tensor_parallel_size为卡数
#engine = VllmEngine(model_path,model_type='qwen2_5_vl',gpu_memory_utilization=0.9,limit_mm_per_prompt={"image": 1},tensor_parallel_size=2)
processor = AutoProcessor.from_pretrained(model_path)

vllm_qwen = LLM(
        model=model_path,
        max_num_seqs=1,
        gpu_memory_utilization=0.9,limit_mm_per_prompt={"image": 1},tensor_parallel_size=2
)

In [8]:
from qwen_vl_utils import process_vision_info
def vllm_qwen_ocr_cn(img_path):
    prompt_ocr="你是一个OCR专家，请提取以下图片中的所有文字内容，并原样返回。"
    messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": img_path,
                    },
                    {"type": "text", "text": prompt_ocr},
                ],
            }
    ]

    prompt = processor.apply_chat_template(messages,
                                           tokenize=False,
                                           add_generation_prompt=True)
    stop_token_ids = None
    if process_vision_info is None:
        image_data = [fetch_image(url) for url in image_urls]
    else:
        image_data, _ = process_vision_info(messages)

    sampling_params = SamplingParams(temperature=0.0,
                                max_tokens=10240,
                                stop_token_ids=stop_token_ids)

    outputs = vllm_qwen.generate(
            {
                "prompt": prompt,
                "multi_modal_data": {
                    "image": image_data
                },
        },
            sampling_params=sampling_params)

    for o in outputs:
        generated_text = o.outputs[0].text
    return generated_text

In [ ]:
res = vllm_qwen_ocr_cn("/data/coding/patent_qa/test/pdf_img/CN1031288C/1.jpg")

In [ ]:
# 进行ocr
import fitz  # PyMuPDF
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
base_dir = '/data/coding/patent_qa/test/'
pdf_file_list = [x for x in os.listdir(base_dir+'/pdf_img/')]
# 前400个，排序
pdf_file_list = sorted(pdf_file_list)[:400]

for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_img/'+file_name) if 'jpg' in x] # pdf文件存储的jpg
    os.makedirs(base_dir+'/pdf_ocr/'+file_name,exist_ok=True)
    for k in tqdm(range(len(file_list))):
        img_path = base_dir+'/pdf_img/'+file_name + '/' + file_list[k]
        page_num = int(file_list[k].split('.')[0])
        # ocr
        response = vllm_qwen_ocr_cn(img_path)
        # 保存ocr结果为txt
        with open(base_dir+f'/pdf_ocr/{file_name}/{page_num}.txt', 'w', encoding='utf-8') as text_file:
            text_file.write(response)
    print(file_name,'  ok')
    with open(base_dir+'test_ocr_finished_list_1.txt', 'a') as f:
        f.write(file_name + '\n')

In [ ]:
# 进行ocr
import fitz  # PyMuPDF
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
base_dir = '/data/coding/patent_qa/test/'
pdf_file_list = [x for x in os.listdir(base_dir+'/pdf_img/')]
# 前400个，排序
pdf_file_list = sorted(pdf_file_list)[400:]
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_img/'+file_name) if 'jpg' in x] # pdf文件存储的jpg
    os.makedirs(base_dir+'/pdf_ocr/'+file_name,exist_ok=True)
    for k in tqdm(range(len(file_list))):
        img_path = base_dir+'/pdf_img/'+file_name + '/' + file_list[k]
        page_num = int(file_list[k].split('.')[0])
        # ocr
        response = vllm_qwen_ocr_cn(img_path)
        # 保存ocr结果为txt
        with open(base_dir+f'/pdf_ocr/{file_name}/{page_num}.txt', 'w', encoding='utf-8') as text_file:
            text_file.write(response)
    print(file_name,'  ok')
    with open(base_dir+'test_ocr_finished_list_2.txt', 'a') as f:
        f.write(file_name + '\n')

# 图片存入向量库

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["MAX_PIXELS"] = '1229312' # 1003520
from gme_inference import GmeQwen2VL
gme = GmeQwen2VL(model_name='/data/coding/llm_model/iic/gme-Qwen2-VL-7B-Instruct',max_image_tokens=1280)

Loading checkpoint shards: 100%|██████████| 9/9 [00:02<00:00,  3.07it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
import os
import pandas as pd
import numpy as np
import tqdm
from warnings import filterwarnings
# 过滤掉一些警告
filterwarnings("ignore")
base_dir = '/data/coding/patent_qa/test/'
pdf_file_list = [x for x in os.listdir(base_dir+'/pdf_img/')]
files_total_cnt = 0
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_img/'+file_name) if 'jpg' in x]
    files_total_cnt +=len(file_list)

In [ ]:
from tqdm import trange,tqdm
# 文件有多个，每个都需要存映射关系
img_page_num_list = []
img_name_list = []
img_vectors = np.empty((files_total_cnt, 3584)) # 向量维度是3584维
idx = 0
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_img/'+file_name) if 'jpg' in x] # pdf文件存储的jpg
    for k in trange(len(file_list)):  
        image_path = base_dir+'/pdf_img/'+file_name + '/' + file_list[k]
        e_text = gme.get_image_embeddings(images=[image_path])
        img_vectors[idx] = e_text[0].to('cpu').numpy()
        page_num = int(file_list[k].split('.')[0])
        img_page_num_list.append(page_num)
        img_name_list.append(file_name)
        idx+=1
# 映射关系存储到pandas里面比较方便
img_page_num_mapping = pd.DataFrame({'index': range(len(img_page_num_list)), 'page_num': img_page_num_list, 'file_name': img_name_list})
# 将向量和页码映射关系存储到文件
np.save('test_pdf_img_vectors.npy', img_vectors)
img_page_num_mapping.to_csv('test_pdf_img_page_num_mapping.csv', index=False) # 存储映射关系

# ocr的结果存入向量库

In [3]:
import os
import pandas as pd
import numpy as np
import tqdm
from warnings import filterwarnings
# 过滤掉一些警告
filterwarnings("ignore")
base_dir = '/data/coding/patent_qa/test/'
pdf_file_list = [x for x in os.listdir(base_dir+'/pdf_img/')]
files_total_cnt = 0
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_ocr/'+file_name) if 'txt' in x]
    files_total_cnt +=len(file_list)

In [5]:
# 文件有多个，每个都需要存映射关系
ocr_page_num_list = []
ocr_name_list = []
ocr_vectors = np.empty((files_total_cnt, 3584)) # 向量维度是3584维
idx = 0
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_ocr/'+file_name) if 'txt' in x] # pdf文件存储的jpg
    for k in range(len(file_list)):  
        text_path = base_dir+'/pdf_ocr/'+file_name + '/' + file_list[k]
        with open(text_path, 'r', encoding='utf-8') as f:
            text_content = f.read()
        e_text = gme.get_text_embeddings(texts=[text_content])
        ocr_vectors[idx] = e_text[0].to('cpu').numpy()
        page_num = int(file_list[k].split('.')[0])
        ocr_page_num_list.append(page_num)
        ocr_name_list.append(file_name)
        idx+=1
# 映射关系存储到pandas里面比较方便
ocr_page_num_mapping = pd.DataFrame({'index': range(len(ocr_page_num_list)), 'page_num': ocr_page_num_list, 'file_name': ocr_name_list})
# 将向量和页码映射关系存储到文件
np.save('test_ocr_vectors.npy', ocr_vectors)
ocr_page_num_mapping.to_csv('test_ocr_page_num_mapping.csv', index=False) # 存储映射关系


encode: 100%|██████████| 1/1 [00:00<00:00, 29.48it/s]


# 读取问题，生成问题的向量

In [7]:
df_question = pd.read_json('/data/coding/patent_qa/test/questions.jsonl',lines=True)
# 问题的vector进行保存
question_vectors = np.empty((len(df_question), 3584))
for i in range(len(df_question)):
    question = df_question.loc[i,'question']
    document_name = df_question.loc[i,'document']
    options = df_question.loc[i,'options']
    full_question = question + ' '.join(options)
    query_vec = gme.get_text_embeddings(texts=[full_question])
    question_vectors[i] = query_vec[0].to('cpu').numpy()
# 保存问题的向量
np.save('all_test_question_vectors.npy', question_vectors)

encode:   0%|          | 0/1 [00:00<?, ?it/s]

encode: 100%|██████████| 1/1 [00:00<00:00, 29.22it/s]


In [12]:
import numpy as np
import pandas as pd
base_dir = '/data/coding/patent_qa/test/'
df_question = pd.read_json("/data/coding/patent_qa/test/questions.jsonl",lines=True)
quesion_vector = np.load('all_test_question_vectors.npy')
test_ocr_page_num_mapping = pd.read_csv('test_ocr_page_num_mapping.csv')
test_ocr_vectors = np.load('test_ocr_vectors.npy')

In [4]:
# index	page_num	file_name
test_pdf_image_vectors_1 = np.load("test_pdf_img_vectors1.npy") # 前400个
test_pdf_image_page_num_mapping_1 = pd.read_csv('test_pdf_img_page_num_mapping1.csv') # 前400个
test_pdf_image_vectors_2 = np.load("test_pdf_img_vectors2.npy") # 后400个
test_pdf_image_page_num_mapping_2 = pd.read_csv('test_pdf_img_page_num_mapping2.csv') # 后400个

In [5]:
test_pdf_image_vectors = np.concatenate([test_pdf_image_vectors_1,test_pdf_image_vectors_2])
test_pdf_image_page_num_mapping_2['index']=test_pdf_image_page_num_mapping_2['index']+test_pdf_image_vectors_1.shape[0]
test_pdf_image_page_num_mapping = pd.concat([test_pdf_image_page_num_mapping_1,test_pdf_image_page_num_mapping_2])

In [ ]:
from swift.llm import InferEngine, InferRequest, PtEngine, RequestConfig, load_dataset
from swift.plugin import InferStats
from swift.llm import VllmEngine
from vllm import LLM, SamplingParams
import os
os.environ['VIDEO_MAX_PIXELS'] = '50176'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["MAX_PIXELS"] = "1229312"
os.environ['FPS_MAX_FRAMES'] = "2"
model_path = "/data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct"
engine = VllmEngine(model_path,model_type='qwen2_5_vl',gpu_memory_utilization=0.9,limit_mm_per_prompt={"image": 2},max_model_len=8192)
def infer_batch(engine, infer_requests):
    request_config = RequestConfig(max_tokens=1024, temperature=0)
    metric = InferStats()
    resp_list = engine.infer(infer_requests, request_config)
    response = resp_list[0].choices[0].message.content
    return response


/data/miniconda/envs/swift/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO:swift] Successfully registered `/data/miniconda/envs/swift/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`.


INFO 06-02 18:42:18 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-02 18:42:18 [__init__.py:239] Automatically detected platform cuda.


2025-06-02 18:42:20,456	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
[INFO:swift] Loading the model using model_dir: /data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct
[INFO:swift] Setting torch_dtype: torch.bfloat16
[WARNING:swift] Please install the package: `pip install "decord" -U`.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
[INFO:swift] Setting image_factor: 28. You can adjust this hyperparameter through the environment variable: `IMAGE_FACTOR`.
[INFO:swift] Setting min_pixels: 3136. You can adjust this hyperparameter through the environment variable: `MIN_PIXELS`.
[INFO:swift] Using environment variable

INFO 06-02 18:42:28 [config.py:717] This model supports multiple tasks: {'embed', 'classify', 'score', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 06-02 18:42:28 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='/data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct', speculative_config=None, tokenizer='/data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=F

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:01,  3.36it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:01<00:02,  1.21it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:02<00:02,  1.03s/it]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:04<00:01,  1.11s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:05<00:00,  1.16s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:05<00:00,  1.05s/it]



INFO 06-02 18:42:35 [loader.py:458] Loading weights took 5.45 seconds
INFO 06-02 18:42:36 [model_runner.py:1140] Model loading took 15.6271 GiB and 5.703531 seconds
WARNING 06-02 18:42:40 [model_runner.py:1305] Computed max_num_seqs (min(256, 8192 // 49152)) to be less than 1. Setting it to the minimum value of 1.
WARNING 06-02 18:42:44 [profiling.py:245] The sequence length used for profiling (max_num_batched_tokens / max_num_seqs = 8192) is too short to hold the multi-modal embeddings in the worst case (49152 tokens in total, out of which {'image': 32768, 'video': 16384} are reserved for multi-modal embeddings). This may cause certain multi-modal inputs to fail during inference, even when the input text is short. To avoid this, you should increase `max_model_len`, reduce `max_num_seqs`, and/or reduce `mm_counts`.
INFO 06-02 18:42:59 [worker.py:287] Memory profiling takes 23.37 seconds
INFO 06-02 18:42:59 [worker.py:287] the current vLLM instance can use total_gpu_memory (39.38GiB) x 

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:19<00:00,  1.76it/s]

INFO 06-02 18:43:22 [model_runner.py:1592] Graph capturing finished in 20 secs, took 0.28 GiB
INFO 06-02 18:43:22 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 46.53 seconds


WARNING 06-02 18:43:22 [sampling_params.py:347] temperature 1e-06 is less than 0.01, which may cause numerical errors nan or inf in tensors. We have maxed it out to 0.01.


In [54]:
# 给定了图片的情况下，有图片本身，也需要召回对应的文本
def get_similar_text_embedding(base_dir,document_name,question_idx,top_k):
    document_name = df_question.document[question_idx].split('.')[0]
    vec_idx = test_ocr_page_num_mapping[test_ocr_page_num_mapping['file_name']==document_name]['index'].values
    candidate_vec = test_ocr_vectors[vec_idx]
    query_vec = quesion_vector[question_idx]
    cos_sim = np.dot(candidate_vec, query_vec) / (np.linalg.norm(candidate_vec) * np.linalg.norm(query_vec))
    # 获取最相似的top_k个索引
    top_k_indices = np.argsort(cos_sim)[-top_k:][::-1]
    retrived_idx = vec_idx[top_k_indices] # 最相近的top_k个
    retrived_page_num = test_ocr_page_num_mapping.loc[retrived_idx]['page_num'].to_list()
    text_list = []
    for i in range(len(retrived_page_num)):
        text_file = base_dir + '/pdf_ocr/' + document_name + '/' + str(retrived_page_num[i]) +'.txt'
        with open(text_file,'r',encoding='utf-8') as f:
            text_list.append(f.read())
    return text_list # 返回一个list,大小最大为top_k


def get_similar_image_embedding(base_dir,document_name,question_idx,top_k):
    document_name = df_question.document[question_idx].split('.')[0]
    vec_idx = test_pdf_image_page_num_mapping[test_pdf_image_page_num_mapping['file_name']==document_name]['index'].values
    candidate_vec = test_pdf_image_vectors[vec_idx]
    query_vec = quesion_vector[question_idx]
    cos_sim = np.dot(candidate_vec, query_vec) / (np.linalg.norm(candidate_vec) * np.linalg.norm(query_vec))
    # 获取最相似的top_k个索引
    top_k_indices = np.argsort(cos_sim)[-top_k:][::-1]
    retrived_idx = vec_idx[top_k_indices] # 最相近的top_k个
    retrived_page_num = test_pdf_image_page_num_mapping.loc[retrived_idx]['page_num'].to_list()
    image_list = []
    for i in range(len(retrived_page_num)):
        image_file = base_dir + '/pdf_img/' + document_name + '/' + str(retrived_page_num[i]) +'.jpg'
        image_list.append(image_file)
    return image_list # 返回一个list,大小最大为top_k

def get_text_answer_vl(document_name,question,options,quesion_idx):
    prompt="你是一个专利内容分析专家，请根据我提供的专利内容回答我的单选题。\n"
    question = "【我的问题】【"
    question += (question +"】\n")
    question += "【选项】【"
    question += (' '.join(options) + "】\n")
    question += ("请在分析我提供的专利问题后回答我的单选题，回答选项字母。专利内容为：\n")
    retrived_list = get_similar_text_embedding(base_dir,document_name,quesion_idx,2)
    question += '\n'.join(retrived_list)
    question += ("请你分析专利内容后，回答我的单选题，回答选项字母，你的答案为：\n")
    #sampling_params = SamplingParams(temperature=0)
    #outputs = text_qa_llm.generate(question,sampling_params)
    #return outputs[0].outputs[0].text
    messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                ],
            }
        ]
    data = dict()
    data['messages'] = messages
    infer_requests = [InferRequest(**data)]
    response = infer_batch(engine, infer_requests)
    return response

def get_image_answer(document_name,question,options,quesion_idx):
    prompt="你是一个专利内容分析专家，请根据我提供的专利内容回答我的单选题。\n"
    question1 = "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("请在分析我提供的专利问题后回答我的单选题，回答选项字母。专利内容为：\n")
    retrived_list = get_similar_image_embedding(base_dir,document_name,quesion_idx,2)
    question2 = ("请你分析专利内容后，回答我的单选题，回答选项字母，你的答案为：\n")
    if len(retrived_list)>1:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {
                        "type": "image",
                        "image": retrived_list[1],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    else:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": retrived_list[0],
                    },
                    {"type": "text", "text": question2},
                ],
            }
        ]
    data = dict()
    data['messages'] = messages
    infer_requests = [InferRequest(**data)]
    response = infer_batch(engine, infer_requests)
    return response

def get_mix_answer(document_name,pic_page_num,question,options,question_idx):
    prompt="你是一个专利内容分析专家，请根据我提供的专利内容回答我的单选题。\n"
    question1 = "【我的问题】【"
    question1 += (question +"】\n")
    question1 += "【选项】【"
    question1 += (' '.join(options) + "】\n")
    question1 += ("请在分析我提供的专利问题后回答我的单选题，回答选项字母。专利内容为：\n")
    retrived_list = get_similar_text_embedding(base_dir,document_name,question_idx,2)
    question2 = ("请你分析专利内容后，回答我的单选题，回答选项字母，你的答案为：\n")
    messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question1},
                    {
                        "type": "image",
                        "image": base_dir + '/pdf_img/' + document_name.split('.')[0]+ '/' + str(pic_page_num) +'.jpg',
                    },
                    {"type": "text", "text": '\n'.join(retrived_list)},
                    {"type": "text", "text": question2},
                ],
            }
    ]
    data = dict()
    data['messages'] = messages
    infer_requests = [InferRequest(**data)]
    response = infer_batch(engine, infer_requests)
    return response

In [55]:
import re
from tqdm import trange
import json
# 实时把结果写入jsonl文件
for i in trange(len(df_question)):
    question = df_question.loc[i,'question']
    document_name = df_question.loc[i,'document']
    options = df_question.loc[i,'options']
    #true_answer = df_question.loc[i,'answer']
    full_question = question + ' '.join(options)
    answer = ''
    text_answer = ''
    image_answer = ''
    if "第" in question and "页" in question and "图": # 问题含有图片
        pic_page_num = re.findall(r"第(\d+)页", question)[0]
        pic_page_num = int(pic_page_num)
        answer = get_mix_answer(document_name,pic_page_num,question,options,i)
    else:
        text_answer = get_text_answer_vl(document_name,question,options,i) # 使用文本来回答
        image_answer = get_image_answer(document_name,question,options,i) # 使用图像来回答
    
    result_dict= dict()
    result_dict['idx'] = str(i)
    result_dict['document'] = document_name
    result_dict['question'] = question
    result_dict['options'] = options
    result_dict['pure_pic_answer'] = answer if answer else ''
    result_dict['text_answer'] = text_answer if text_answer else ''
    result_dict['image_answer'] = image_answer if image_answer else ''
    with open('test_set_output_by_qwen25_vl.jsonl', 'a', encoding='utf-8') as f:
        f.write(json.dumps(result_dict, ensure_ascii=False) + '\n')


100%|██████████| 1/1 [00:02<00:00,  2.34s/it]
